In [1]:
# import platform
# print(platform.architecture())
import pyodbc
import datetime
from datetime import datetime, timedelta


In [2]:

# Connection details (replace with your credentials)
db2_odbc_conn_string = 'DSN=DB2PRD-64;UID=TFC;PWD=TFC1010'
mssql_conn_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=10.2.9.99;DATABASE=ROBOTIC;UID=nirun.n;PWD=Fec@08824'
# Connect to DB2 database using ODBC
# Execute the SQL statement (assuming a database connection is established):
mssql_conn = pyodbc.connect(mssql_conn_string)
mssql_cursor = mssql_conn.cursor()


In [3]:
# Get the current date and time
current_date = datetime.now() - timedelta(days=2)
 

# Format the date as 'YYYYMMDD'
formatted_date = current_date.strftime('%Y%m%d')
# Extract the month from the formatted date
cur_date_Month = formatted_date[4:6]  # Slicing from index 4 to 6 (excluding index 6)
# Extract the month from the formatted date
cur_date_Day = formatted_date[6:8]  # Slicing from index 4 to 6 (excluding index 6)

# Print or return the formatted date
# print(formatted_date)

In [4]:
db2_conn = pyodbc.connect(db2_odbc_conn_string)
db2_cursor = db2_conn.cursor()
# sql_str = "select werks, mblnr, BUDAT_MKPF, CPUTM_MKPF,	matnr, BWART, lgort, MENGE, meins, charg, aufnr, MJAHR, ZEILE   from MSEG where mandt = '900' and BUDAT_MKPF = '" + formatted_date +"' AND LGORT = 'A217' and (BWART = '101' OR BWART = '102')"
sql_str = "select werks, mblnr, BUDAT_MKPF, CPUTM_MKPF,	matnr, BWART, lgort, MENGE, meins, charg, aufnr, MJAHR, ZEILE   from MSEG where mandt = '900' and BUDAT_MKPF = '" + '20240817' +"' AND LGORT = 'A217' and (BWART = '101' OR BWART = '102')"

db2_cursor.execute(sql_str)   
db2_data = db2_cursor.fetchall()
# Close DB2 connection
db2_cursor.close()
db2_conn.close()


In [8]:
db2_data

[('4100', '4905909644', '20240816', '080740', '000000300000006073', '101', 'A217', Decimal('2400.000'), 'ST ', '240816000 ', '421700009016', '2024', '0001'),
 ('4100', '4905909645', '20240816', '080846', '000000300000006077', '101', 'A217', Decimal('2400.000'), 'ST ', '240816000 ', '421700008973', '2024', '0001'),
 ('4100', '4905909646', '20240816', '080923', '000000300000006445', '101', 'A217', Decimal('800.000'), 'ST ', '240816000 ', '421700009028', '2024', '0001'),
 ('4100', '4905909647', '20240816', '081003', '000000300000006449', '101', 'A217', Decimal('800.000'), 'ST ', '240816000 ', '421700009043', '2024', '0001'),
 ('4100', '4905909651', '20240816', '081308', '000000300000006062', '101', 'A217', Decimal('3750.000'), 'ST ', '240816000 ', '421700009034', '2024', '0001'),
 ('4100', '4905909652', '20240816', '081325', '000000300000006063', '101', 'A217', Decimal('3250.000'), 'ST ', '240816000 ', '421700009005', '2024', '0001'),
 ('4100', '4905909653', '20240816', '081339', '0000003

In [9]:
import socket

# Get the computer name
computer_name = socket.gethostname()

# Get the local IP address
local_ip = socket.gethostbyname(socket.gethostname())

# print(f"Local IP Address: {local_ip}")
# print(f"Computer Name: {computer_name}")

from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()
current_datetime 

datetime.datetime(2024, 8, 18, 18, 55, 19, 206513)

In [10]:
# # Connect to MSSQL database
# mssql_conn = pyodbc.connect(mssql_conn_string)
# mssql_cursor = mssql_conn.cursor()
# mssql_cursor.execute("DELETE FROM DM_D_MATGRP1_TABLE")  # Replace with actual column names
# mssql_conn.commit()
# mssql_cursor.close()
# mssql_conn.close()

lv_sql = "" 
i = 0
for row in db2_data:
    i += 1
    # Convert each column to string and strip whitespace
    trimmed_row = tuple(str(column).strip() for column in row)
    # print(trimmed_row)
    # Construct the desired string with parentheses and commas:
      
    
    lv_sql += "(SYSDATETIME(), '" + computer_name + "', '" + local_ip + "', '" + trimmed_row[0] + "', '" + trimmed_row[1] 
    lv_sql += "', CONVERT(VARCHAR(10), CAST('" + trimmed_row[2] + "' AS DATE), 120) ,"  
    lv_sql += "CONVERT(VARCHAR(8), CAST(STUFF(STUFF('" + trimmed_row[3] + "' , 3, 0, ':'), 6, 0, ':') AS TIME), 108),'"    
    lv_sql +=  trimmed_row[4].lstrip('0') + "', '" + trimmed_row[5] + "', '" + trimmed_row[6] + "', '" 
    if trimmed_row[5] == '101':
        lv_sql += trimmed_row[7]  
    else:
        lv_sql += '-' + trimmed_row[7]  
    lv_sql += "', '" + trimmed_row[8] + "', '" + trimmed_row[9] 
    lv_sql += "', '" + trimmed_row[10] + "', '" + trimmed_row[11] + "', '" + trimmed_row[1]+trimmed_row[12] 
    lv_sql += "','" + cur_date_Month + "','" +  cur_date_Day + "'),"
    # CreateOperatedTime,Computer_Name,Computer_IP,0:Plant,1:Material Document, 2:Posting Date, 3:Time of Entry, 4:Material, 5:Movement Type, 6:Storage Location, 7:Quantity, 8:Base Unit of Measure, 
    # 9:Batch, 10: Order, 11:Year, 12:Item
    # print(lv_sql)

lv_sql = lv_sql[:-1]
# print(i)  

In [12]:

lv_sqlinsert = "INSERT INTO  DM_D_MATGRP1_TABLE (LOADDATE,SPRAS,MATERIAL_GRP1,MATERIAL_GRP1_DES) VALUES "
lv_sqlinsert = '''insert into SPIMMB51Semi 
                    (CreateOperatedTime,ComputerName,ComputerIP,Plant, MaterialDocument, PostingDate, TimeofEntry, Material, MovementType, StorageLocation, Quantity
                    , BaseUnitofMeasure,  Batch, PrdOrder, MatdocYear, Item, Month, Day)
                    values
               '''
lv_sqlinsert += lv_sql
lv_sqlinsert

# Execute the SQL statement (assuming a database connection is established):
mssql_conn = pyodbc.connect(mssql_conn_string)
mssql_cursor = mssql_conn.cursor()
mssql_cursor.execute(lv_sqlinsert)
# Commit changes and close MSSQL connection
mssql_conn.commit()
mssql_cursor.close()
mssql_conn.close()

# Execute the SQL statement (assuming a database connection is established):
mssql_conn = pyodbc.connect(mssql_conn_string)
mssql_cursor = mssql_conn.cursor()
lv_sqlinsert = '''UPDATE SPIMMB51Semi
                    SET 
                        PostOperatedTime = CAST(PostingDate AS DATETIME) + CAST(TimeofEntry AS DATETIME)
                    WHERE PostOperatedTime IS NULL
                '''
mssql_cursor.execute(lv_sqlinsert)
# Commit changes and close MSSQL connection
mssql_conn.commit()
mssql_cursor.close()
mssql_conn.close()
print("ETL process completed successfully on table DM_D_MATGRP1_TABLE with ",i, " records.")